

__Set your own route planner for a given OSM map and GTFS file and query server on localhost for a detailed routes__


### OTP walkthrough

from: http://docs.opentripplanner.org/en/latest/Basic-Tutorial/

1. Install Java SDK
2. Download [latest](https://repo1.maven.org/maven2/org/opentripplanner/otp/1.4.0/otp-1.4.0-shaded.jar) .jar from [Maven](https://repo1.maven.org/maven2/org/opentripplanner/otp/1.4.0/)
3. Download gtfs.zip of a given city into a `folder` from `transit.land`
4. Download osm.pgf of your area from [here](https://www.interline.io/osm/extracts/) (need to login to get API token)
5. call from terminal `java -Xmx6G -jar otp-1.4.0-shaded.jar --build data_path --inMemory` changing `6G` as memory allocated (6GB) and `data_path` to your path with osm and gtfs files
6. if succesful you shall be able to open a web search in your browser: `http://localhost:8080/`
7. and query it via API, for instance: 'http://localhost:8080/otp/routers/default/plan?fromPlace=52.25177%2C20.92758&toPlace=52.20550%2C21.00517&time=1%3A02pm&date=04-09-2020&mode=TRANSIT%2CWALK&maxWalkDistance=5000&arriveBy=false'


In [1]:
import pandas as pd
import requests

local imports

In [2]:
from main import make_query, parse_OTP_response
from utils import plot

set up

In [3]:
PATH = 'georequests.csv'
OTP_API = "http://localhost:8080/otp/routers/default/plan"

input

In [4]:
df = pd.read_csv(PATH, index_col=[0])  # load the csv
df.treq = pd.to_datetime(df.treq)
df.sample(5)[['origin_x','origin_y','destination_x','destination_y','treq']]

,origin_x,origin_y,destination_x,destination_y,treq
923,4.919858,52.354815,4.876652,52.362379,2020-10-29 14:33:05
944,4.956360,52.397104,4.872823,52.366161,2020-10-29 14:30:04
174,4.886725,52.383804,4.882440,52.370075,2020-10-29 14:58:38
175,4.780391,52.359392,4.884474,52.361284,2020-10-29 15:09:21
447,4.859886,52.344896,4.877036,52.366819,2020-10-29 14:47:48


In [5]:
df1 = pd.DataFrame(columns=['origin_x','origin_y','destination_x','destination_y','treq'])
df1['origin_x'] = [16.89423, 16.91294, 16.94984]
df1['origin_y'] = [52.40504, 52.42797, 52.41404]

df1['destination_x'] = [16.94984, 16.92324, 16.88410]
df1['destination_y'] = [52.40619, 52.39833, 52.36973]

df1['treq'] = pd.Timestamp(2023, 3, 7, 12)

df =df1

sample query (before you can query the server you need to run it -see run OTP server.ipynb in this folder)

[query synthax](http://dev.opentripplanner.org/apidoc/1.0.0/resource_PlannerResource.html)  
[response json structure](http://dev.opentripplanner.org/apidoc/1.0.0/json_Response.html)

In [6]:
row = df.sample(1).squeeze()
query = make_query(row)
r = requests.get(OTP_API, params=query)
route = parse_OTP_response(r.json())
route

{'success': True,
 'n_itineraries': 3,
 'duration': 1904,
 'walkDistance': 899.6474060001171,
 'transfers': 1,
 'transitTime': 1140,
 'waitingTime': 2,
 'modes': [['WALK', 400, 470],
  ['TRAM', 840, 3768],
  ['TRAM', 300, 1390],
  ['WALK', 362, 428]]}

#### visualization

In [47]:
plot(r.json()['plan'], color = 'green')

Trip from (16.8942,52.4050) to (16.9498,52.4062) at 1678186800000. 
3 connections found. 
Best one is 32min (1038m walk, 0 transfer(s), wait time 0.03min)

LEG 	 MODE 	DIST 	TIME
-----------------------------
1	WALK	438	361
2	TRAM	5010	1080
3	WALK	600	496


output

In [22]:
pd.read_csv('georequests_PT.csv', index_col=[0]).sample(5)

,success,n_itineraries,duration,walkDistance,transfers,transitTime,waitingTime,modes
id,,,,,,,,
619,True,3,1089,539.499970,0,623,2,"[['WALK', 51, 67], ['TRAM', 623, 2238], ['WALK..."
20,True,3,1063,127.663805,0,936,2,"[['WALK', 35, 28], ['TRAM', 936, 3922], ['WALK..."
756,True,3,2021,1163.640707,0,1076,2,"[['WALK', 314, 392], ['BUS', 1076, 4792], ['WA..."
926,True,3,1795,797.164368,1,1140,21,"[['WALK', 278, 344], ['TRAM', 822, 3694], ['WA..."
502,True,3,1088,871.273308,0,373,2,"[['WALK', 295, 356], ['TRAM', 373, 1886], ['WA..."


full response

In [21]:
r.json()['requestParameters']

{'date': '10-29-2020',
 'mode': 'TRANSIT,WALK',
 'arriveBy': 'false',
 'fromPlace': '52.3502454,4.9443192',
 'toPlace': '52.3650389,4.8738375',
 'time': '03:28pm',
 'maxWalkDistance': '2000'}

---